I will try to implement every technique I know to clean up the data.

In [67]:
import pandas as pd

from pandas import read_csv,read_json

Grab the data from the online repository

In [89]:
df_original=read_csv("https://raw.githubusercontent.com/Ikea-179/Hateful-Meme-Detection/main/data/new_merged.csv")

In [90]:
df_original.sample(10)

,Unnamed: 0.1,Unnamed: 0,id,Fulltext,text,img,label,clean text
5300,5300,5300,35408,do you know how to use an oven! or do i need t...,do you know how to use an oven! or do i need t...,img/35408.png,1,do you know how to use an oven! or do i need t...
1034,1034,1034,63517,they will have a surgery soon Woman Woman Woma...,they will have a surgery soon,img/63517.png,0,they will have a surgery soon Female Female Fe...
3950,3950,3950,17290,when my autistic son finds out we're having ch...,when my autistic son finds out we're having ch...,img/17290.png,1,when my autistic son finds out were having chi...
5632,5632,5632,35217,me getting euthanized Girl Woman Boy Girl Pers...,me getting euthanized,img/35217.png,0,me getting euthanized Person Male Male Male Ma...
42,42,42,70193,that chiken is so black no one is going to eat...,that chiken is so black no one is going to eat it,img/70193.png,0,that chiken is so black no one is going to eat...
5651,5651,5651,26784,black people: *exist* lmm: it's free real esta...,black people: *exist* lmm: it's free real estate,img/26784.png,1,black people: *exist* lmm: its free real estat...
12109,12109,13309,68304,show me the ape son Man Man Boy Male Female Fe...,show me the ape son,img/68304.png,0,show me the ape sonMale Female Female White Wh...
4824,4824,4824,14028,"i'm an evolved monkey on a cartoon ball, nasa ...","i'm an evolved monkey on a cartoon ball, nasa ...",img/14028.png,0,"im an evolved monkey on a cartoon ball, nasa t..."
11562,11562,12762,13540,ladies and gents i present old navy's goat col...,ladies and gents i present old navy's goat col...,img/13540.png,0,ladies and gents i present old navys goat coll...
7304,7304,7304,75408,if they raise theirs and destroy our flag are ...,if they raise theirs and destroy our flag are ...,img/75408.png,0,if they raise theirs and destroy our flag are ...


All the dependencies

In [ ]:
import nltk
import regex as re
import numpy as np
import autocorrect as ac        #https://github.com/filyp/autocorrect

from tqdm.auto import tqdm,trange  #https://tqdm.github.io
from autocorrect import Speller
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from gensim.parsing.preprocessing import remove_stopwords

#pd.options.mode.chained_assignment=None
min_string_len=12
max_string_len=200

stem=PorterStemmer()

In [92]:
def autocorrect_dataframe(df):
    spell=Speller(lang='en',only_replacements=True) #only_replacements=True for OCR, otherwise false
    for i in trange(len(df),desc='Correcting Data'):
        df.at[i,'text_ac']=spell(df.at[i,'text'])
    return df

In [93]:
def clean_string(string): #function to clean a string | to be used in clean_csv
    string=string.lower() #convert string to lowercase
    string=re.sub('\n|\r|\t','',string) #cleaning of whitespaces
    string=re.sub(r'[^\w\s]+','',string) #cleaning of symbols
    return string

In [94]:
def stemming(string): #churi kora
    result = re.sub('[^a-zA-Z]',' ',string)
    result = result.lower()
    result = result.split()
    result = [stem.stem(word) for word in result if not word in stopwords.words('english')]
    result = ' '.join(result)
    return result

In [112]:
def clean_dataframe(df): #cleaning the dataframe
    df=df.dropna() #drop all tuples with empty attributes
    length=[]
    [length.append(len(str(text))) for text in tqdm(df["text"],desc='Measuring length')] #calculate length of an article
    df['length']=length #saves the length of the article in a new column
    df=df.drop(df['text'][df['length']<min_string_len].index,axis=0) #drop all tuples with less than min_string_len letters
    df=df.drop(df['text'][df['length']>max_string_len].index,axis=0) #drop all tuples with greater than max_string_len letters
    #df['text']=df['title']+" "+df['text'] #append title to text
    df.reset_index(drop=True,inplace=True) #reset all indexes of the dataset
    df=autocorrect_dataframe(df)
    vfunc=np.vectorize(clean_string) #vectorize the clean_string function for use in pandas df
    df['text_ac']=vfunc(df['text_ac']) #apply the vectorized function to further clean the data
    for i in trange(len(df),desc='Cleaning Data'):
        df.at[i,'text_s']=remove_stopwords(df.at[i,'text_ac']) #cleaning stopwords
        df.at[i,'text_ss']=stemming(df.at[i,'text_s']) #stemming, comment this line if accuracy drops
    return df

In [113]:
df_cleaned=clean_dataframe(df_original)
df_cleaned.sample(10)

Measuring length:   0%|          | 0/10087 [00:00<?, ?it/s]

Correcting data:   0%|          | 0/9925 [00:00<?, ?it/s]

Cleaning Data:   0%|          | 0/9925 [00:00<?, ?it/s]

,Unnamed: 0.1,Unnamed: 0,id,Fulltext,text,img,label,clean text,text_ac,length,text_s,text_ss
3927,3993,3993,65073,jews and americans will ruin the world [['Jew...,jews and americans will ruin the world,img/65073.png,1,jews and americans will ruin the worldJewish A...,jews and americans will ruin the world,38,jews americans ruin world,jew american ruin world
6029,6126,6126,51629,that reminds me! it's time to fuck the goat! M...,that reminds me! it's time to fuck the goat!,img/51629.png,1,that reminds me! its time to fuck the goat!Mal...,that reminds me its time to fuck the goat,44,reminds time fuck goat,remind time fuck goat
1018,1032,1032,13958,"""i'd like a book on suicide."" ""fuck off. you w...","""i'd like a book on suicide."" ""fuck off. you w...",img/13958.png,1,"""id like a book on suicide."" ""fuck off. you wo...",id like a book on suicide fuck off you wont br...,66,id like book suicide fuck wont bring,id like book suicid fuck wont bring
4354,4426,4426,35802,it's gettin hot in here so take off all ur clo...,it's gettin hot in here so take off all ur clo...,img/35802.png,1,its gettin hot in here so take off all ur clot...,its retain hot in here so take off all ur clothes,50,retain hot ur clothes,retain hot ur cloth
3266,3316,3316,78139,i don't know what she's fixing but mine just b...,i don't know what she's fixing but mine just b...,img/78139.png,0,i dont know what shes fixing but mine just bro...,i dont know what shes fixing but mine just broke,50,dont know shes fixing broke,dont know she fix broke
4079,4147,4147,94065,if yall daughters dancing like this.. they can...,if yall daughters dancing like this.. they can...,img/94065.png,0,if yall daughters dancing like this.. they can...,if call daughters dancing like this they cant ...,63,daughters dancing like play mines,daughter danc like play mine
105,111,111,96578,mohammad is fucked by blink hacker group! oink...,mohammad is fucked by blink hacker group! oink...,img/96578.png,1,mohammad is fucked by blink hacker group! oink...,mohammad is sucked by blink hacker group link ...,94,mohammad sucked blink hacker group link link h...,mohammad suck blink hacker group link link hi ...
6382,6482,6482,92154,what kind of punch can take out 20 kids and 6 ...,what kind of punch can take out 20 kids and 6 ...,img/92154.png,1,what kind of punch can take out 20 kids and 6 ...,what kind of punch can take out 20 kids and 6 ...,66,kind punch 20 kids 6 adults sandy hook,kind punch kid adult sandi hook
9384,9537,10737,40856,hey! stop picking on him! f*ggots are people t...,hey! stop picking on him! f*ggots are people too!,img/40856.png,1,hey! stop picking on him! f*ggots are people t...,hey stop picking on him fgross are people too,49,hey stop picking fgross people,hey stop pick fgross peopl
9919,10081,11281,96015,we're so excited that we're finally able to be...,we're so excited that we're finally able to be...,img/96015.png,0,were so excited that were finally able to be o...,were so excited that were finally able to be o...,56,excited finally able,excit final abl


In [84]:
df_cleaned.loc[df_cleaned['length'].idxmax(axis=0,skipna=True)]

Unnamed: 0.1                                                 3850
Unnamed: 0                                                   3850
id                                                          60789
Fulltext        musslamic democrat, ilhan omar, has threatened...
text            musslamic democrat ilhan omar has threatened m...
img                                                 img/60789.png
label                                                           0
clean text      musslamic democrat, ilhan omar, has threatened...
text_ac         musslamic democrat, clean omar, has threatened...
length                                                        200
text_s          musslamic democrat, clean omar, threatened mem...
text_ss         musslam democrat clean omar threaten member co...
Name: 3789, dtype: object

In [85]:
df_cleaned.loc[df_cleaned['length'].idxmin(axis=0,skipna=True)]

Unnamed: 0.1                                                   47
Unnamed: 0                                                     47
id                                                          86354
Fulltext        officer down  [['Icon', 'Vector', 'vector art ...
text                                                 officer down
img                                                 img/86354.png
label                                                           0
clean text              officer downIcon, Vector, vector art dead
text_ac                                              officer down
length                                                         12
text_s                                                    officer
text_ss                                                     offic
Name: 44, dtype: object

In [88]:
df_cleaned_2=df_cleaned[['id','text','text_ac','text_s','text_ss','label']]
df_cleaned_2.sample(10)

,id,text,text_ac,text_s,text_ss,label
8799,14389,democrats were new bold and we hate you alexan...,"democrats we're new, bold, and we hate you! al...","democrats we're new, bold, hate you! alexandri...",democrat new bold hate alexandria ocasio corte...,1
1903,20435,kids playing in the backyard,kids playing in the backyard,kids playing backyard,kid play backyard,0
6739,67129,i only wear silk panties cotton ones remind me...,i only wear silk parties cotton ones remind me...,wear silk parties cotton ones remind slavery,wear silk parti cotton one remind slaveri,1
10763,13475,when youre only a little racist goonie,when you're only a little racist google,you're little racist google,littl racist googl,0
5228,43570,mondays got me like,monkeys got me like,monkeys got like,monkey got like,0
6885,5612,of all time the most underrated cook,of all time the most underrated cook,time underrated cook,time underr cook,0
9763,89250,when you start burning calories instead of people,when you start burning calories instead of people,start burning calories instead people,start burn calori instead peopl,1
8229,20578,you said islam is violent i kill you,you said islam is violent? i kill you!,said islam violent? kill you!,said islam violent kill,1
3034,90673,when you ask the jewish girl for her number an...,when you ask the jewish girl for her number an...,ask jewish girl number starts rolling sleeve,ask jewish girl number start roll sleev,1
3115,20386,america in reality dont let the media divide us,america in reality don't let the media divide us!,america reality don't let media divide us!,america realiti let media divid us,0
